In [1]:
import numpy as np
import pandas as pd
import xgboost

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
#This is my data cleaning code I set up in Quick_and_Dirty.ipynb

data = pd.read_csv('data/application_train.csv')
test_data = pd.read_csv('data/application_test.csv')
data.rename(columns={x:x.lower() for x in data.columns}, inplace=True)
test_data.rename(columns={x:x.lower() for x in test_data.columns}, inplace=True)

float_columns = [x for x in data.columns if data[x].dtype == np.float64]
float_columns_with_nulls =  [c for c in float_columns if data[c].isna().values.any()]
categorical_columns = [x for x in data.columns if data[x].dtype == object]

#Cleaning training data 

#adding a null category for each categorical column 
for col in categorical_columns:
    data.loc[:,col] = data[col].apply(lambda x: x if not pd.isna(x) else "null")
    
uniques = {col:data[col].unique() for col in categorical_columns}

#imputing nans in float columns and adding an is_imputed column
for col in float_columns_with_nulls:
    null_col = col+'_is_imputed'
    data[null_col] = data[col].apply(lambda x: 1 if pd.isna(x) else 0)
    data.loc[:,col] = data[col].apply(lambda x: 0 if pd.isna(x) else x )
    
#creating dummy columns for all the categorical columns
for col in categorical_columns:
    for cat in uniques[col]:
        data[col+'_'+cat] = data[col].apply(lambda x: 1 if x == cat else 0)
    data= data.drop(col, axis = 1)
    
### Cleaning test data    
#adding a null category for each categorical column 
for col in categorical_columns:
    test_data.loc[:,col] = test_data[col].apply(lambda x: x if not pd.isna(x) else "null")
    

#imputing nans in float columns and adding an is_imputed column
for col in float_columns_with_nulls:
    null_col = col+'_is_imputed'
    test_data[null_col] = test_data[col].apply(lambda x: 1 if pd.isna(x) else 0)
    test_data.loc[:,col] = test_data[col].apply(lambda x: 0 if pd.isna(x) else x )
    
#creating dummy columns for all the categorical columns
for col in categorical_columns:
    for cat in uniques[col]:
        test_data[col+'_'+cat] = test_data[col].apply(lambda x: 1 if x == cat else 0)
    test_data= test_data.drop(col, axis = 1)


In [3]:
X,y = data.drop(['target','sk_id_curr'],axis = 1), data['target'] 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=17)

In [ ]:
booster = xgboost.XGBClassifier(max_depth = 4, )